# Voxel Grid Creation

## Purpose

This notebook teaches you how to create and configure voxel grids for the AM-QADF framework. You'll learn to create uniform, adaptive, and multi-resolution grids with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Create and configure voxel grids interactively
- ✅ Understand coordinate systems and transformations
- ✅ Use adaptive and multi-resolution grids
- ✅ Visualize grid structures in 2D and 3D
- ✅ Configure grid properties and metadata

## Estimated Duration

45-60 minutes

---

## Overview

Voxel grids are the foundation of the AM-QADF framework. They provide a structured 3D representation of spatial data:

- 🧊 **Uniform Grids**: Fixed resolution across the entire domain
- 🎯 **Adaptive Grids**: Variable resolution based on data density
- 📊 **Multi-Resolution Grids**: Hierarchical grids with multiple resolution levels

Use the interactive widgets below to create and explore voxel grids - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

# Try to import voxelization classes
VOXEL_AVAILABLE = False
try:
    from am_qadf.voxelization.voxel_grid import VoxelGrid
    VOXEL_AVAILABLE = True
except ImportError:
    print("⚠️ VoxelGrid not available - using demo mode")

ADAPTIVE_AVAILABLE = False
try:
    from am_qadf.voxelization.adaptive_resolution import AdaptiveResolutionGrid
    ADAPTIVE_AVAILABLE = True
except ImportError:
    print("⚠️ AdaptiveResolutionGrid not available")

MULTI_AVAILABLE = False
try:
    from am_qadf.voxelization.multi_resolution import MultiResolutionGrid
    MULTI_AVAILABLE = True
except ImportError:
    print("⚠️ MultiResolutionGrid not available")

# Try to import query client for model selection
STL_CLIENT_AVAILABLE = False
try:
    from am_qadf.query import STLModelClient
    STL_CLIENT_AVAILABLE = True
except ImportError as e:
    STL_CLIENT_AVAILABLE = False
    print(f"⚠️ STLModelClient not available: {e}")

# Try to import alignment storage for loading aligned data
ALIGNMENT_STORAGE_AVAILABLE = False
alignment_storage = None
try:
    from am_qadf.synchronization import AlignmentStorage
    ALIGNMENT_STORAGE_AVAILABLE = True
except ImportError as e:
    ALIGNMENT_STORAGE_AVAILABLE = False
    print(f"⚠️ AlignmentStorage not available: {e}")

# Initialize MongoDB connection (optional, for model selection)
mongo_client = None
stl_client = None

if INFRASTRUCTURE_AVAILABLE and STL_CLIENT_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        if mongo_client and mongo_client.is_connected():
            stl_client = STLModelClient(mongo_client=mongo_client)
            # Initialize alignment storage if available
            if ALIGNMENT_STORAGE_AVAILABLE:
                try:
                    alignment_storage = AlignmentStorage(mongo_client=mongo_client)
                    print("✅ MongoDB connection established")
                    print("✅ AlignmentStorage initialized")
                except Exception as e:
                    print("✅ MongoDB connection established")
                    print(f"⚠️ AlignmentStorage initialization failed: {e}")
            else:
                print("✅ MongoDB connection established")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")

# Try to import voxel grid storage (after mongo_client is initialized)
STORAGE_AVAILABLE = False
voxel_storage = None

try:
    # Import directly from voxel_storage to avoid importing voxel_domain_client
    # which has dependencies on signal_mapping that may not be available
    import importlib.util
    voxel_storage_path = src_dir / 'am_qadf' / 'voxel_domain' / 'voxel_storage.py'
    
    if voxel_storage_path.exists():
        spec = importlib.util.spec_from_file_location("voxel_storage", voxel_storage_path)
        voxel_storage_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(voxel_storage_module)
        VoxelGridStorage = voxel_storage_module.VoxelGridStorage
        STORAGE_AVAILABLE = True
        
        # Initialize storage only if mongo_client is available
        if mongo_client and mongo_client.is_connected():
            try:
                voxel_storage = VoxelGridStorage(mongo_client)
                print("✅ VoxelGridStorage initialized")
            except Exception as e:
                print(f"⚠️ Error initializing VoxelGridStorage: {e}")
                import traceback
                traceback.print_exc()
        else:
            print("⚠️ VoxelGridStorage available but MongoDB not connected")
    else:
        raise ImportError(f"voxel_storage.py not found at {voxel_storage_path}")
        
except Exception as e:
    STORAGE_AVAILABLE = False
    print(f"⚠️ VoxelGridStorage not available: {e}")
    # Try fallback to regular import
    try:
        from am_qadf.voxel_domain.voxel_storage import VoxelGridStorage
        STORAGE_AVAILABLE = True
        if mongo_client and mongo_client.is_connected():
            voxel_storage = VoxelGridStorage(mongo_client)
            print("✅ VoxelGridStorage initialized (fallback import)")
    except Exception as e2:
        print(f"⚠️ Fallback import also failed: {e2}")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 69600ffe4f114c10b6db4d1f, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


Failed to initialize MongoDB connection: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 69600ffe4f114c10b6db4d1f, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


⚠️ VoxelGridStorage available but MongoDB not connected
✅ Setup complete!


## Interactive Voxel Grid Creator

Use the widgets below to create and configure voxel grids. Select grid type, set bounding box and resolution, configure coordinate systems, and visualize the results!


In [2]:
# Create Interactive Voxel Grid Interface

# Global state
current_grid = None
current_grid_type = None
saved_grids = {}
current_grid_id = None  # Track the saved grid ID
current_grid_model_name = None  # Track the model name for current grid
current_grid_model_id = None  # Track the model ID for current grid
loaded_aligned_data = None  # Store loaded aligned data for grid creation
current_alignment_id = None  # Track the alignment ID used for grid creation
current_alignment_model_id = None  # Track the model ID from alignment
current_alignment_model_name = None  # Track the model name from alignment

# ============================================
# Top Panel: Grid Type and Actions
# ============================================

grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type = RadioButtons(
    options=[('Uniform', 'uniform'), ('Adaptive', 'adaptive'), ('Multi-Resolution', 'multi')],
    value='uniform',
    description='Type:',
    style={'description_width': 'initial'}
)

create_button = Button(
    description='Create Grid',
    button_style='success',
    icon='plus',
    layout=Layout(width='120px')
)

load_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

save_button = Button(
    description='Save Grid',
    button_style='',
    icon='save',
    layout=Layout(width='120px')
)

top_panel = HBox([
    grid_type_label,
    grid_type,
    create_button,
    load_button,
    save_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Grid Configuration
# ============================================

# Bounding Box Section
bbox_label = widgets.HTML("<b>Bounding Box:</b>")
bbox_mode = RadioButtons(
    options=[('From Alignment', 'alignment'), ('From Model', 'model'), ('Custom', 'custom'), ('Interactive', 'interactive')],
    value='alignment',  # Default to 'alignment' to use aligned data
    description='Mode:',
    style={'description_width': 'initial'}
)

# Alignment selector (for From Alignment mode)
alignment_options = [("━━━ Select Alignment ━━━", None), ("━━━ All Alignments ━━━", "ALL")]

if alignment_storage and mongo_client:
    try:
        alignments = alignment_storage.list_alignments(limit=100)
        # Group by model for easier selection
        for align in alignments:
            model_id = align.get('model_id', 'Unknown')
            model_name = align.get('model_name', 'Unknown')
            align_id = align.get('alignment_id', 'Unknown')
            align_mode = align.get('alignment_mode', 'both')
            created = align.get('created_at', '')
            if isinstance(created, str):
                created_str = created[:10] if len(created) > 10 else created
            else:
                created_str = str(created)[:10] if hasattr(created, '__str__') else ''
            alignment_options.append(
                (f"{model_name} - {align_mode} ({created_str})", align_id)
            )
        if len(alignment_options) == 2:  # Only "Select" and "All" options
            alignment_options.append(("No alignments available", None))
    except Exception as e:
        print(f"⚠️ Error loading alignments: {e}")
        alignment_options.append(("Error loading alignments", None))
else:
    alignment_options.append(("Alignment storage not available", None))

# Set default alignment selection
default_alignment_value = None
if len(alignment_options) > 2:  # More than just "Select" and "All"
    # Default to first alignment (skip "Select Alignment" option)
    default_alignment_value = alignment_options[2][1] if len(alignment_options) > 2 else None

alignment_dropdown = Dropdown(
    options=alignment_options,
    value=default_alignment_value,
    description='Alignment:',
    style={'description_width': 'initial'},
    layout=Layout(display='flex', width='auto')  # Show by default since mode is 'alignment'
)

# Model selector (for From Model mode)
# Load models from MongoDB
models = []
model_options = [("━━━ Select Model ━━━", None), ("━━━ All Models ━━━", "ALL")]

if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
        if len(model_options) == 2:  # Only "Select" and "All" options
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))
else:
    # Demo mode
    model_options.extend([
        ("Demo Model 1", "demo-001"),
        ("Demo Model 2", "demo-002")
    ])

# Set default model selection
default_model_value = None
if len(model_options) > 2:  # More than just "Select" and "All"
    # Default to "All Models" if available, otherwise first model
    if any(opt[1] == "ALL" for opt in model_options):
        default_model_value = "ALL"
    else:
        # Use first actual model (skip "Select Model" option)
        default_model_value = model_options[2][1] if len(model_options) > 2 else None
elif len(model_options) == 2 and any(opt[1] == "ALL" for opt in model_options):
    default_model_value = "ALL"

model_dropdown = Dropdown(
    options=model_options,
    value=default_model_value,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(display='flex', width='auto')  # Show by default since mode is 'model'
)

# Custom bounding box sliders
bbox_x_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='X Min (mm):', style={'description_width': 'initial'})
bbox_x_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='X Max (mm):', style={'description_width': 'initial'})
bbox_y_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Min (mm):', style={'description_width': 'initial'})
bbox_y_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Max (mm):', style={'description_width': 'initial'})
bbox_z_min = FloatSlider(value=0.0, min=-1000.0, max=1000.0, step=1.0, description='Z Min (mm):', style={'description_width': 'initial'})
bbox_z_max = FloatSlider(value=100.0, min=-1000.0, max=1000.0, step=1.0, description='Z Max (mm):', style={'description_width': 'initial'})

bbox_sliders = VBox([
    bbox_x_min, bbox_x_max,
    bbox_y_min, bbox_y_max,
    bbox_z_min, bbox_z_max
])

def update_bbox_controls(change):
    """Show/hide bounding box controls based on mode."""
    if change['new'] == 'alignment':
        alignment_dropdown.layout.display = 'flex'
        model_dropdown.layout.display = 'none'
        bbox_sliders.layout.display = 'none'
        # Update bbox sliders when alignment is selected
        if alignment_dropdown.value and alignment_dropdown.value != "ALL":
            update_bbox_from_alignment()
    elif change['new'] == 'model':
        alignment_dropdown.layout.display = 'none'
        model_dropdown.layout.display = 'flex'
        bbox_sliders.layout.display = 'none'
        # Update bbox sliders when model is selected
        if model_dropdown.value and model_dropdown.value != "ALL":
            update_bbox_from_model()
    elif change['new'] == 'custom':
        alignment_dropdown.layout.display = 'none'
        model_dropdown.layout.display = 'none'
        bbox_sliders.layout.display = 'flex'
    else:  # interactive
        alignment_dropdown.layout.display = 'none'
        model_dropdown.layout.display = 'none'
        bbox_sliders.layout.display = 'none'

def update_bbox_from_alignment():
    """Update bounding box sliders from selected alignment's aligned data."""
    if alignment_dropdown.value and alignment_dropdown.value != "ALL" and alignment_storage:
        try:
            # Load alignment with aligned data
            alignment = alignment_storage.load_alignment(alignment_dropdown.value, load_aligned_data=True)
            if alignment and 'aligned_data' in alignment:
                aligned_data = alignment['aligned_data']
                
                # Collect all points from all sources
                all_points = []
                for source_name, source_data in aligned_data.items():
                    if isinstance(source_data, dict) and 'points' in source_data:
                        points = source_data['points']
                        if isinstance(points, np.ndarray) and len(points) > 0:
                            all_points.append(points)
                
                if all_points:
                    # Combine all points
                    combined_points = np.vstack(all_points)
                    # Calculate bounding box from aligned data
                    bbox_min = combined_points.min(axis=0)
                    bbox_max = combined_points.max(axis=0)
                    
                    # Update sliders
                    bbox_x_min.value = float(bbox_min[0])
                    bbox_x_max.value = float(bbox_max[0])
                    bbox_y_min.value = float(bbox_min[1])
                    bbox_y_max.value = float(bbox_max[1])
                    bbox_z_min.value = float(bbox_min[2])
                    bbox_z_max.value = float(bbox_max[2])
        except Exception as e:
            print(f"⚠️ Error updating bbox from alignment: {e}")

def update_bbox_from_model():
    """Update bounding box sliders from selected model."""
    if model_dropdown.value and model_dropdown.value != "ALL" and stl_client:
        try:
            model_data = stl_client.get_model(model_dropdown.value)
            if model_data and 'metadata' in model_data:
                bbox = model_data['metadata'].get('bounding_box', {})
                if bbox and 'min' in bbox and 'max' in bbox:
                    bbox_min = bbox['min']
                    bbox_max = bbox['max']
                    # Update sliders (but don't trigger events)
                    bbox_x_min.value = bbox_min[0]
                    bbox_x_max.value = bbox_max[0]
                    bbox_y_min.value = bbox_min[1]
                    bbox_y_max.value = bbox_max[1]
                    bbox_z_min.value = bbox_min[2]
                    bbox_z_max.value = bbox_max[2]
        except Exception as e:
            print(f"⚠️ Error updating bbox from model: {e}")

# Also update bbox when alignment selection changes
def on_alignment_change(change):
    """Handle alignment selection change."""
    if bbox_mode.value == 'alignment' and change['new'] and change['new'] != "ALL":
        update_bbox_from_alignment()

alignment_dropdown.observe(on_alignment_change, names='value')

# Also update bbox when model selection changes
def on_model_change(change):
    """Handle model selection change."""
    if bbox_mode.value == 'model' and change['new'] and change['new'] != "ALL":
        update_bbox_from_model()
    elif bbox_mode.value == 'model' and change['new'] == "ALL":
        # Update bbox for "All Models" case
        if stl_client and models:
            all_bboxes = []
            for m in models:
                model_id = m.get('model_id')
                if model_id:
                    try:
                        model_data = stl_client.get_model(model_id)
                        if model_data and 'metadata' in model_data:
                            bbox = model_data['metadata'].get('bounding_box', {})
                            if bbox and 'min' in bbox and 'max' in bbox:
                                all_bboxes.append((bbox['min'], bbox['max']))
                    except Exception as e:
                        print(f"⚠️ Error fetching model {model_id}: {e}")
            
            if all_bboxes:
                # Find union of all bounding boxes
                min_coords = [min(bbox[0][i] for bbox in all_bboxes) for i in range(3)]
                max_coords = [max(bbox[1][i] for bbox in all_bboxes) for i in range(3)]
                # Update sliders
                bbox_x_min.value = min_coords[0]
                bbox_x_max.value = max_coords[0]
                bbox_y_min.value = min_coords[1]
                bbox_y_max.value = max_coords[1]
                bbox_z_min.value = min_coords[2]
                bbox_z_max.value = max_coords[2]

model_dropdown.observe(on_model_change, names='value')

bbox_mode.observe(update_bbox_controls, names='value')

        # Initialize bounding box from default selection
if bbox_mode.value == 'alignment' and default_alignment_value:
    # Update bbox from default alignment
    update_bbox_from_alignment()
elif bbox_mode.value == 'model' and default_model_value:
    if default_model_value == "ALL":
        on_model_change({'new': "ALL"})
    else:
        update_bbox_from_model()

bbox_section = VBox([
    bbox_label,
    bbox_mode,
    alignment_dropdown,
    model_dropdown,
    bbox_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Resolution Section
resolution_label = widgets.HTML("<b>Resolution:</b>")
resolution_mode = RadioButtons(
    options=[('Uniform', 'uniform'), ('Per-Axis', 'per_axis'), ('Adaptive', 'adaptive')],
    value='uniform',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Uniform resolution
uniform_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Resolution (mm):', style={'description_width': 'initial'})

# Per-axis resolution
x_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='X Res (mm):', style={'description_width': 'initial'})
y_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Y Res (mm):', style={'description_width': 'initial'})
z_resolution = FloatSlider(value=1.0, min=0.01, max=10.0, step=0.01, description='Z Res (mm):', style={'description_width': 'initial'})

per_axis_sliders = VBox([
    x_resolution, y_resolution, z_resolution
], layout=Layout(display='none'))

# Adaptive resolution config (collapsible)
adaptive_expand = Checkbox(value=False, description='Show Adaptive Config', style={'description_width': 'initial'})
adaptive_strategy = Dropdown(
    options=[('Data Density', 'density'), ('Gradient', 'gradient'), ('Custom Map', 'custom')],
    value='density',
    description='Strategy:',
    style={'description_width': 'initial'}
)

adaptive_config = VBox([
    adaptive_expand,
    adaptive_strategy
], layout=Layout(display='none'))

def update_resolution_controls(change):
    """Show/hide resolution controls based on mode."""
    if change['new'] == 'uniform':
        uniform_resolution.layout.display = 'flex'
        per_axis_sliders.layout.display = 'none'
        adaptive_config.layout.display = 'none'
    elif change['new'] == 'per_axis':
        uniform_resolution.layout.display = 'none'
        per_axis_sliders.layout.display = 'flex'
        adaptive_config.layout.display = 'none'
    else:  # adaptive
        uniform_resolution.layout.display = 'none'
        per_axis_sliders.layout.display = 'none'
        adaptive_config.layout.display = 'flex' if adaptive_expand.value else 'none'

resolution_mode.observe(update_resolution_controls, names='value')

def update_adaptive_display(change):
    """Show/hide adaptive config when expanded."""
    if resolution_mode.value == 'adaptive':
        adaptive_config.layout.display = 'flex' if change['new'] else 'none'

adaptive_expand.observe(update_adaptive_display, names='value')

resolution_section = VBox([
    resolution_label,
    resolution_mode,
    uniform_resolution,
    per_axis_sliders,
    adaptive_config
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Coordinate System Section
coord_label = widgets.HTML("<b>Coordinate System:</b>")
coord_system = Dropdown(
    options=[('Machine', 'machine'), ('Part', 'part'), ('Build Platform', 'build'), ('Custom', 'custom')],
    value='build',
    description='System:',
    style={'description_width': 'initial'}
)

origin_x = FloatText(value=0.0, description='Origin X:', style={'description_width': 'initial'})
origin_y = FloatText(value=0.0, description='Origin Y:', style={'description_width': 'initial'})
origin_z = FloatText(value=0.0, description='Origin Z:', style={'description_width': 'initial'})

rotation_x = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot X (deg):', style={'description_width': 'initial'})
rotation_y = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot Y (deg):', style={'description_width': 'initial'})
rotation_z = FloatSlider(value=0.0, min=-180.0, max=180.0, step=1.0, description='Rot Z (deg):', style={'description_width': 'initial'})

transform_preview = Button(description='Preview Transform', button_style='', layout=Layout(width='150px'))

coord_section = VBox([
    coord_label,
    coord_system,
    origin_x, origin_y, origin_z,
    rotation_x, rotation_y, rotation_z,
    transform_preview
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Grid Properties Section
props_label = widgets.HTML("<b>Grid Properties:</b>")
aggregation_method = Dropdown(
    options=[('mean', 'mean'), ('max', 'max'), ('min', 'min'), ('sum', 'sum'), ('median', 'median')],
    value='mean',
    description='Aggregation:',
    style={'description_width': 'initial'}
)

sparse_storage = Checkbox(value=True, description='Sparse Storage', style={'description_width': 'initial'})
compression = Checkbox(value=False, description='Compression', style={'description_width': 'initial'})

props_section = VBox([
    props_label,
    aggregation_method,
    sparse_storage,
    compression
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Left panel assembly
left_panel = VBox([
    bbox_section,
    resolution_section,
    coord_section,
    props_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Grid Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('3D View', '3d'), ('2D Slices', '2d'), ('Properties', 'props')],
    value='props',
    description='View:',
    style={'description_width': 'initial'}
)

# 3D viewer output
viewer_3d = Output(layout=Layout(height='400px', overflow='auto'))

# 2D slice controls
slice_axis = RadioButtons(
    options=[('XY', 'xy'), ('XZ', 'xz'), ('YZ', 'yz')],
    value='xy',
    description='Axis:',
    style={'description_width': 'initial'}
)

slice_position = IntSlider(value=0, min=0, max=100, step=1, description='Position:', style={'description_width': 'initial'})
slice_viewer = Output(layout=Layout(height='400px'))

slice_controls = VBox([
    slice_axis,
    slice_position,
    slice_viewer
], layout=Layout(display='none'))

# Properties display
props_display = Output(layout=Layout(height='400px', overflow='auto'))

def update_viz_display(change):
    """Show/hide visualization controls based on mode."""
    if change['new'] == '3d':
        viewer_3d.layout.display = 'flex'
        slice_controls.layout.display = 'none'
        props_display.layout.display = 'none'
    elif change['new'] == '2d':
        viewer_3d.layout.display = 'none'
        slice_controls.layout.display = 'flex'
        props_display.layout.display = 'none'
    else:  # props
        viewer_3d.layout.display = 'none'
        slice_controls.layout.display = 'none'
        props_display.layout.display = 'flex'

viz_mode.observe(update_viz_display, names='value')

center_panel = VBox([
    widgets.HTML("<h3>Grid Visualization</h3>"),
    viz_mode,
    viewer_3d,
    slice_controls,
    props_display
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Grid Statistics & Actions
# ============================================

# Grid Statistics
stats_label = widgets.HTML("<b>Grid Statistics:</b>")
stats_display = widgets.HTML("No grid created yet")
stats_section = VBox([
    stats_label,
    stats_display
], layout=Layout(padding='5px'))

# Grid Metadata
metadata_label = widgets.HTML("<b>Grid Metadata:</b>")
metadata_display = widgets.HTML("No metadata available")
metadata_section = VBox([
    metadata_label,
    metadata_display
], layout=Layout(padding='5px'))

# Quick Actions
actions_label = widgets.HTML("<b>Quick Actions:</b>")
export_button = Button(description='Export Grid', button_style='', icon='download', layout=Layout(width='150px'))
validate_button = Button(description='Validate Grid', button_style='', icon='check', layout=Layout(width='150px'))
compare_button = Button(description='Compare Grids', button_style='', icon='copy', layout=Layout(width='150px'))

actions_section = VBox([
    actions_label,
    export_button,
    validate_button,
    compare_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    stats_section,
    metadata_section,
    actions_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to create grid")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
error_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Grid Creation Functions
# ============================================

def create_grid(button):
    """Create grid based on current widget settings."""
    global current_grid, current_grid_type
    
    import time
    
    status_display.value = "<b>Status:</b> Creating grid..."
    progress_bar.value = 0
    error_display.value = ""
    
    # Show warning if using model mode instead of alignment mode
    if bbox_mode.value == 'model':
        error_display.value = "<span style='color: orange;'>⚠️ Using STL model bounding boxes. For aligned data, select 'From Alignment' mode.</span>"
    elif bbox_mode.value == 'alignment':
        error_display.value = "<span style='color: green;'>✅ Using aligned data for bounding box calculation.</span>"
    
    try:
        # Get bounding box
        status_display.value = "<b>Status:</b> Fetching bounding box..."
        progress_bar.value = 10
        time.sleep(0.1)  # Small delay for visual feedback
        
        # Global variable to store loaded aligned data for potential use
        global loaded_aligned_data, current_alignment_id, current_alignment_model_id, current_alignment_model_name
        loaded_aligned_data = None
        current_alignment_id = None
        current_alignment_model_id = None
        current_alignment_model_name = None
        
        if bbox_mode.value == 'alignment' and alignment_dropdown.value:
            # Load bounding box from aligned data
            status_display.value = "<b>Status:</b> Loading bounding box from alignment..."
            progress_bar.value = 15
            time.sleep(0.1)
            
            if alignment_dropdown.value == "ALL":
                # For "All Alignments", use union of all aligned data bounding boxes
                if alignment_storage:
                    try:
                        all_alignments = alignment_storage.list_alignments(limit=100)
                        all_bboxes = []
                        for align in all_alignments:
                            align_id = align.get('alignment_id')
                            if align_id:
                                try:
                                    alignment_data = alignment_storage.load_alignment(align_id, load_aligned_data=True)
                                    if alignment_data and 'aligned_data' in alignment_data:
                                        aligned_data = alignment_data['aligned_data']
                                        # Collect all points
                                        all_points = []
                                        for source_name, source_data in aligned_data.items():
                                            if isinstance(source_data, dict) and 'points' in source_data:
                                                points = source_data['points']
                                                if isinstance(points, np.ndarray) and len(points) > 0:
                                                    all_points.append(points)
                                        if all_points:
                                            combined_points = np.vstack(all_points)
                                            bbox_min = combined_points.min(axis=0)
                                            bbox_max = combined_points.max(axis=0)
                                            all_bboxes.append((bbox_min, bbox_max))
                                except Exception as e:
                                    print(f"⚠️ Error loading alignment {align_id}: {e}")
                        
                        if all_bboxes:
                            min_coords = [min(bbox[0][i] for bbox in all_bboxes) for i in range(3)]
                            max_coords = [max(bbox[1][i] for bbox in all_bboxes) for i in range(3)]
                            bbox_min = tuple(min_coords)
                            bbox_max = tuple(max_coords)
                        else:
                            # Fallback
                            bbox_min = (-50.0, -50.0, 0.0)
                            bbox_max = (50.0, 50.0, 100.0)
                    except Exception as e:
                        error_display.value = f"<span style='color: orange;'>⚠️ Error loading alignments: {e}. Using default bbox.</span>"
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
            else:
                # Single alignment selected
                align_id = alignment_dropdown.value
                if alignment_storage and align_id:
                    try:
                        alignment = alignment_storage.load_alignment(align_id, load_aligned_data=True)
                        if alignment and 'aligned_data' in alignment:
                            loaded_aligned_data = alignment['aligned_data']
                            current_alignment_id = align_id
                            current_alignment_model_id = alignment.get('model_id')
                            current_alignment_model_name = alignment.get('model_name', 'Unknown')
                            
                            # Collect all points from all sources
                            all_points = []
                            for source_name, source_data in loaded_aligned_data.items():
                                if isinstance(source_data, dict) and 'points' in source_data:
                                    points = source_data['points']
                                    if isinstance(points, np.ndarray) and len(points) > 0:
                                        all_points.append(points)
                            
                            if all_points:
                                # Combine all points
                                combined_points = np.vstack(all_points)
                                # Calculate bounding box from aligned data
                                bbox_min = tuple(combined_points.min(axis=0))
                                bbox_max = tuple(combined_points.max(axis=0))
                            else:
                                # Fallback if no points found
                                bbox_min = (-50.0, -50.0, 0.0)
                                bbox_max = (50.0, 50.0, 100.0)
                        else:
                            error_display.value = "<span style='color: orange;'>⚠️ No aligned data found in alignment. Using default bbox.</span>"
                            bbox_min = (-50.0, -50.0, 0.0)
                            bbox_max = (50.0, 50.0, 100.0)
                    except Exception as e:
                        error_display.value = f"<span style='color: orange;'>⚠️ Error loading alignment: {e}. Using default bbox.</span>"
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
        elif bbox_mode.value == 'model' and model_dropdown.value:
            if model_dropdown.value == "ALL":
                # For "All Models", use union of all bounding boxes
                if stl_client and models:
                    all_bboxes = []
                    for m in models:
                        model_id = m.get('model_id')
                        if model_id:
                            try:
                                model_data = stl_client.get_model(model_id)
                                if model_data and 'metadata' in model_data:
                                    bbox = model_data['metadata'].get('bounding_box', {})
                                    if bbox and 'min' in bbox and 'max' in bbox:
                                        all_bboxes.append((bbox['min'], bbox['max']))
                            except Exception as e:
                                print(f"⚠️ Error fetching model {model_id}: {e}")
                    
                    if all_bboxes:
                        # Find union of all bounding boxes
                        min_coords = [min(bbox[0][i] for bbox in all_bboxes) for i in range(3)]
                        max_coords = [max(bbox[1][i] for bbox in all_bboxes) for i in range(3)]
                        bbox_min = tuple(min_coords)
                        bbox_max = tuple(max_coords)
                    else:
                        # Fallback
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    # Demo mode fallback
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
            else:
                # Single model selected
                model_id = model_dropdown.value
                if stl_client and model_id:
                    try:
                        model_data = stl_client.get_model(model_id)
                        if model_data and 'metadata' in model_data:
                            bbox = model_data['metadata'].get('bounding_box', {})
                            if bbox and 'min' in bbox and 'max' in bbox:
                                bbox_min = tuple(bbox['min'])
                                bbox_max = tuple(bbox['max'])
                            else:
                                # Fallback if bbox not found
                                bbox_min = (-50.0, -50.0, 0.0)
                                bbox_max = (50.0, 50.0, 100.0)
                        else:
                            bbox_min = (-50.0, -50.0, 0.0)
                            bbox_max = (50.0, 50.0, 100.0)
                    except Exception as e:
                        error_display.value = f"<span style='color: orange;'>⚠️ Error fetching model: {e}. Using default bbox.</span>"
                        bbox_min = (-50.0, -50.0, 0.0)
                        bbox_max = (50.0, 50.0, 100.0)
                else:
                    # Demo mode
                    bbox_min = (-50.0, -50.0, 0.0)
                    bbox_max = (50.0, 50.0, 100.0)
        elif bbox_mode.value == 'custom':
            bbox_min = (bbox_x_min.value, bbox_y_min.value, bbox_z_min.value)
            bbox_max = (bbox_x_max.value, bbox_y_max.value, bbox_z_max.value)
        else:
            bbox_min = (-50.0, -50.0, 0.0)
            bbox_max = (50.0, 50.0, 100.0)
        
        status_display.value = "<b>Status:</b> Calculating bounding box..."
        progress_bar.value = 30
        time.sleep(0.1)
        
        # Get resolution
        status_display.value = "<b>Status:</b> Setting resolution..."
        progress_bar.value = 40
        time.sleep(0.1)
        
        if resolution_mode.value == 'uniform':
            resolution = uniform_resolution.value
        elif resolution_mode.value == 'per_axis':
            # Use average for uniform grids, or create per-axis grid if supported
            resolution = (x_resolution.value + y_resolution.value + z_resolution.value) / 3.0
        else:
            resolution = uniform_resolution.value  # Default for adaptive
        
        # Create grid based on type
        status_display.value = f"<b>Status:</b> Creating {grid_type.value} grid..."
        progress_bar.value = 50
        time.sleep(0.2)  # Slightly longer delay for grid creation
        
        grid_type_val = grid_type.value
        current_grid_type = grid_type_val
        
        if grid_type_val == 'uniform' and VOXEL_AVAILABLE:
            current_grid = VoxelGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                resolution=resolution,
                aggregation=aggregation_method.value
            )
        elif grid_type_val == 'adaptive' and ADAPTIVE_AVAILABLE:
            current_grid = AdaptiveResolutionGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                base_resolution=resolution
            )
        elif grid_type_val == 'multi' and MULTI_AVAILABLE:
            current_grid = MultiResolutionGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                base_resolution=resolution,
                num_levels=3,
                level_ratio=2.0
            )
        else:
            # Demo mode - create a simple representation
            class DemoGrid:
                def __init__(self, bbox_min, bbox_max, resolution):
                    self.bbox_min = np.array(bbox_min)
                    self.bbox_max = np.array(bbox_max)
                    self.resolution = resolution
                    self.size = self.bbox_max - self.bbox_min
                    self.dims = np.ceil(self.size / resolution).astype(int)
                    self.dims = np.maximum(self.dims, [1, 1, 1])
                    self.actual_size = self.dims * resolution
                    self.aggregation = aggregation_method.value
                    self.grid_type = grid_type_val
                    self.created_at = datetime.now()
            
            current_grid = DemoGrid(bbox_min, bbox_max, resolution)
        
        status_display.value = "<b>Status:</b> Initializing grid structure..."
        progress_bar.value = 70
        time.sleep(0.2)
        
        status_display.value = "<b>Status:</b> Calculating grid dimensions..."
        progress_bar.value = 80
        time.sleep(0.1)
        
        # Update displays
        status_display.value = "<b>Status:</b> Updating displays..."
        progress_bar.value = 90
        time.sleep(0.1)
        
        update_grid_displays()
        
        progress_bar.value = 100
        time.sleep(0.1)
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Grid created successfully</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error creating grid</span>"
        progress_bar.value = 0

def update_grid_displays():
    """Update all grid-related displays."""
    global current_grid
    
    if current_grid is None:
        return
    
    # Update statistics
    if hasattr(current_grid, 'dims'):
        dims = current_grid.dims
        total_voxels = int(np.prod(dims))
        memory_mb = total_voxels * 8 / (1024 * 1024)  # Rough estimate
        
        stats_html = f"""
        <p><b>Dimensions:</b> {dims[0]} × {dims[1]} × {dims[2]}</p>
        <p><b>Total Voxels:</b> {total_voxels:,}</p>
        <p><b>Memory (est.):</b> {memory_mb:.2f} MB</p>
        <p><b>Resolution:</b> {current_grid.resolution if hasattr(current_grid, 'resolution') else 'N/A'} mm</p>
        """
    else:
        stats_html = "<p>Statistics not available</p>"
    
    stats_display.value = stats_html
    
    # Update metadata
    metadata_html = f"""
    <p><b>Type:</b> {current_grid_type}</p>
    <p><b>Created:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    <p><b>Coordinate System:</b> {coord_system.value}</p>
    <p><b>Aggregation:</b> {aggregation_method.value}</p>
    <p><b>Storage:</b> {'Sparse' if sparse_storage.value else 'Dense'}</p>
    """
    metadata_display.value = metadata_html
    
    # Update properties display
    with props_display:
        clear_output(wait=True)
        
        # Format bounding box values nicely
        def format_bbox(bbox):
            if bbox is None or not hasattr(current_grid, 'bbox_min'):
                return 'N/A'
            if isinstance(bbox, np.ndarray):
                # Format each value with 2 decimal places, using decimal notation
                # For very small numbers, show as 0.00 instead of scientific notation
                formatted = []
                for val in bbox:
                    if abs(val) < 1e-10:  # Essentially zero
                        formatted.append("0.00")
                    elif abs(val) < 0.01:  # Very small but not zero
                        formatted.append(f"{val:.4f}")
                    else:
                        formatted.append(f"{val:.2f}")
                return f"[{', '.join(formatted)}]"
            elif isinstance(bbox, (list, tuple)):
                formatted = []
                for val in bbox:
                    if abs(val) < 1e-10:
                        formatted.append("0.00")
                    elif abs(val) < 0.01:
                        formatted.append(f"{val:.4f}")
                    else:
                        formatted.append(f"{val:.2f}")
                return f"[{', '.join(formatted)}]"
            return str(bbox)
        
        # Format dimensions
        def format_dims(dims):
            if dims is None or not hasattr(current_grid, 'dims'):
                return 'N/A'
            if isinstance(dims, np.ndarray):
                return f"[{', '.join(map(str, dims.astype(int)))}]"
            elif isinstance(dims, (list, tuple)):
                return f"[{', '.join(map(str, map(int, dims)))}]"
            return str(dims)
        
        # Get model name from current selection or saved grid info
        display_model_name = current_grid_model_name
        if not display_model_name and model_dropdown.value and model_dropdown.value != "ALL":
            # Try to get from selected model
            for m in models:
                if m.get('model_id') == model_dropdown.value:
                    display_model_name = m.get('model_name') or m.get('filename') or m.get('original_stem', 'Unknown')
                    break
        
        # Get grid ID (if saved)
        display_grid_id = current_grid_id[:8] + "..." if current_grid_id else "Not saved"
        
        bbox_min_str = format_bbox(current_grid.bbox_min if hasattr(current_grid, 'bbox_min') else None)
        bbox_max_str = format_bbox(current_grid.bbox_max if hasattr(current_grid, 'bbox_max') else None)
        dims_str = format_dims(current_grid.dims if hasattr(current_grid, 'dims') else None)
        resolution_str = f"{current_grid.resolution:.2f}" if hasattr(current_grid, 'resolution') else 'N/A'
        total_voxels = int(np.prod(current_grid.dims)) if hasattr(current_grid, 'dims') else 'N/A'
        
        # Get resolution mode and values
        resolution_mode_str = resolution_mode.value if hasattr(resolution_mode, 'value') else 'N/A'
        if resolution_mode_str == 'uniform':
            resolution_detail = f"{uniform_resolution.value:.2f} mm (uniform)"
        elif resolution_mode_str == 'per_axis':
            resolution_detail = f"X: {x_resolution.value:.2f}, Y: {y_resolution.value:.2f}, Z: {z_resolution.value:.2f} mm"
        else:
            resolution_detail = f"{uniform_resolution.value:.2f} mm (adaptive)"
        
        # Get coordinate system details
        coord_origin = f"[{origin_x.value:.2f}, {origin_y.value:.2f}, {origin_z.value:.2f}]"
        coord_rotation = f"[{rotation_x.value:.1f}°, {rotation_y.value:.1f}°, {rotation_z.value:.1f}°]"
        
        html = f"""
        <h4>Grid Properties</h4>
        <table border='1' style='border-collapse: collapse; width: 100%;'>
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Grid Information</b></th></tr>
        <tr><td><b>Grid ID</b></td><td>{display_grid_id}</td></tr>
        <tr><td><b>Model Name</b></td><td>{display_model_name if display_model_name else 'N/A'}</td></tr>
        <tr><td><b>Model ID</b></td><td>{current_grid_model_id[:8] + '...' if current_grid_model_id else 'N/A'}</td></tr>
        <tr><td><b>Grid Type</b></td><td>{current_grid_type if current_grid_type else 'N/A'}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Spatial Properties</b></th></tr>
        <tr><td>Bounding Box Min</td><td>{bbox_min_str}</td></tr>
        <tr><td>Bounding Box Max</td><td>{bbox_max_str}</td></tr>
        <tr><td>Bounding Box Source</td><td>{bbox_mode.value if hasattr(bbox_mode, 'value') else 'N/A'}</td></tr>
        <tr><td>Dimensions</td><td>{dims_str}</td></tr>
        <tr><td>Total Voxels</td><td>{total_voxels:,}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Resolution Settings</b></th></tr>
        <tr><td>Resolution Mode</td><td>{resolution_mode_str}</td></tr>
        <tr><td>Resolution</td><td>{resolution_detail}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Coordinate System</b></th></tr>
        <tr><td>System Type</td><td>{coord_system.value if hasattr(coord_system, 'value') else 'N/A'}</td></tr>
        <tr><td>Origin</td><td>{coord_origin}</td></tr>
        <tr><td>Rotation</td><td>{coord_rotation}</td></tr>
        
        <tr style='background-color: #e8f4f8;'><th colspan='2'><b>Grid Properties</b></th></tr>
        <tr><td>Aggregation Method</td><td>{aggregation_method.value}</td></tr>
        <tr><td>Sparse Storage</td><td>{'Yes' if sparse_storage.value else 'No'}</td></tr>
        <tr><td>Compression</td><td>{'Yes' if compression.value else 'No'}</td></tr>
        </table>
        """
        display(HTML(html))
    
    # Update 2D slice visualization
    update_slice_view()

def update_slice_view():
    """Update 2D slice visualization."""
    global current_grid
    
    if current_grid is None or not hasattr(current_grid, 'dims'):
        return
    
    with slice_viewer:
        clear_output(wait=True)
        
        dims = current_grid.dims
        axis = slice_axis.value
        pos = slice_position.value
        
        # Create a simple visualization
        fig, ax = plt.subplots(figsize=(8, 6))
        
        if axis == 'xy':
            # XY slice at Z position
            z_idx = int(pos * dims[2] / 100) if dims[2] > 0 else 0
            z_idx = min(z_idx, dims[2] - 1)
            data = np.zeros((dims[1], dims[0]))
            ax.imshow(data, cmap='viridis', origin='lower', aspect='auto')
            ax.set_title(f'XY Slice at Z={z_idx} (Position {pos}%)')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
        elif axis == 'xz':
            # XZ slice at Y position
            y_idx = int(pos * dims[1] / 100) if dims[1] > 0 else 0
            y_idx = min(y_idx, dims[1] - 1)
            data = np.zeros((dims[2], dims[0]))
            ax.imshow(data, cmap='viridis', origin='lower', aspect='auto')
            ax.set_title(f'XZ Slice at Y={y_idx} (Position {pos}%)')
            ax.set_xlabel('X')
            ax.set_ylabel('Z')
        else:  # yz
            # YZ slice at X position
            x_idx = int(pos * dims[0] / 100) if dims[0] > 0 else 0
            x_idx = min(x_idx, dims[0] - 1)
            data = np.zeros((dims[2], dims[1]))
            ax.imshow(data, cmap='viridis', origin='lower', aspect='auto')
            ax.set_title(f'YZ Slice at X={x_idx} (Position {pos}%)')
            ax.set_xlabel('Y')
            ax.set_ylabel('Z')
        
        plt.tight_layout()
        plt.show()

# ============================================
# Save Grid Function
# ============================================

def save_grid(button):
    """Save current grid to MongoDB."""
    global current_grid
    
    if current_grid is None:
        error_display.value = "<span style='color: red;'>❌ No grid to save. Create a grid first.</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>Error: No grid created</span>"
        return
    
    if not voxel_storage or not mongo_client or not mongo_client.is_connected():
        error_display.value = "<span style='color: red;'>❌ MongoDB storage not available. Cannot save grid.</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Storage unavailable</span>"
        return
    
    # Determine which models to save for
    models_to_save = []
    
    # Handle alignment mode - use model from alignment
    if bbox_mode.value == 'alignment' and current_alignment_model_id:
        # Use model info from alignment
        models_to_save = [{
            'model_id': current_alignment_model_id,
            'model_name': current_alignment_model_name or 'Unknown',
            'alignment_id': current_alignment_id
        }]
    elif bbox_mode.value == 'alignment' and alignment_dropdown.value == "ALL":
        # For "All Alignments", get all unique models from alignments
        if alignment_storage:
            try:
                all_alignments = alignment_storage.list_alignments(limit=100)
                seen_models = set()
                for align in all_alignments:
                    model_id = align.get('model_id')
                    if model_id and model_id not in seen_models:
                        seen_models.add(model_id)
                        models_to_save.append({
                            'model_id': model_id,
                            'model_name': align.get('model_name', 'Unknown'),
                            'alignment_id': align.get('alignment_id')
                        })
                if not models_to_save:
                    error_display.value = "<span style='color: red;'>❌ No alignments available.</span>"
                    status_display.value = "<b>Status:</b> <span style='color: red;'>Error: No alignments available</span>"
                    return
            except Exception as e:
                error_display.value = f"<span style='color: red;'>❌ Error loading alignments: {e}</span>"
                status_display.value = "<b>Status:</b> <span style='color: red;'>Error loading alignments</span>"
                return
        else:
            error_display.value = "<span style='color: red;'>❌ Alignment storage not available.</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Alignment storage unavailable</span>"
            return
    elif bbox_mode.value == 'model' and model_dropdown.value == "ALL":
        # Save grid for all models
        if models and len(models) > 0:
            models_to_save = models
        else:
            error_display.value = "<span style='color: red;'>❌ No models available.</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error: No models available</span>"
            return
    elif bbox_mode.value == 'model' and model_dropdown.value and model_dropdown.value != "ALL":
        # Save for single selected model
        model_id = model_dropdown.value
        for m in models:
            if m.get('model_id') == model_id:
                models_to_save = [m]
                break
        if not models_to_save:
            error_display.value = "<span style='color: red;'>❌ Selected model not found.</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Model not found</span>"
            return
    else:
        # No model selected, use first available
        if models and len(models) > 0:
            models_to_save = [models[0]]
            error_display.value = f"<span style='color: orange;'>⚠️ No model selected. Using first model.</span>"
        else:
            error_display.value = "<span style='color: red;'>❌ No model available. Please select a model first.</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error: No model selected</span>"
            return
    
    try:
        saved_count = 0
        total_models = len(models_to_save)
        
        if total_models > 1:
            status_display.value = f"<b>Status:</b> Saving grid for {total_models} models..."
        else:
            status_display.value = "<b>Status:</b> Saving grid to MongoDB..."
        
        progress_bar.value = 0
        
        saved_grid_ids = []
        
        for idx, model in enumerate(models_to_save):
            model_id = model.get('model_id')
            model_name = model.get('model_name') or model.get('filename') or model.get('original_stem', 'Unknown')
            alignment_id = model.get('alignment_id')  # May be None if not from alignment
            
            # Generate unique grid name for each model
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            if alignment_id:
                grid_name = f"grid_{grid_type.value}_{model_name}_aligned_{timestamp}" if total_models > 1 else f"grid_{grid_type.value}_aligned_{timestamp}"
            else:
                grid_name = f"grid_{grid_type.value}_{model_name}_{timestamp}" if total_models > 1 else f"grid_{grid_type.value}_{timestamp}"
            
            try:
                # Update progress
                progress = int((idx / total_models) * 80) + 10
                progress_bar.value = progress
                
                if total_models > 1:
                    status_display.value = f"<b>Status:</b> Creating and saving grid {idx+1}/{total_models} for {model_name}..."
                
                # If this is from alignment and we have alignment_id, create grid from that specific alignment's data
                grid_to_save = current_grid  # Default to current grid
                
                if bbox_mode.value == 'alignment' and alignment_id and alignment_storage:
                    # Load this specific alignment's data and create a grid for it
                    try:
                        alignment = alignment_storage.load_alignment(alignment_id, load_aligned_data=True)
                        if alignment and 'aligned_data' in alignment:
                            aligned_data = alignment['aligned_data']
                            
                            # Collect all points from all sources for this alignment
                            all_points = []
                            for source_name, source_data in aligned_data.items():
                                if isinstance(source_data, dict) and 'points' in source_data:
                                    points = source_data['points']
                                    if isinstance(points, np.ndarray) and len(points) > 0:
                                        all_points.append(points)
                            
                            if all_points:
                                # Combine all points and calculate bounding box
                                combined_points = np.vstack(all_points)
                                bbox_min = tuple(combined_points.min(axis=0))
                                bbox_max = tuple(combined_points.max(axis=0))
                                
                                # Get resolution
                                if resolution_mode.value == 'uniform':
                                    resolution = uniform_resolution.value
                                elif resolution_mode.value == 'per_axis':
                                    resolution = (x_resolution.value + y_resolution.value + z_resolution.value) / 3.0
                                else:
                                    resolution = uniform_resolution.value
                                
                                # Create grid for this specific alignment
                                grid_type_val = grid_type.value
                                if grid_type_val == 'uniform' and VOXEL_AVAILABLE:
                                    grid_to_save = VoxelGrid(
                                        bbox_min=bbox_min,
                                        bbox_max=bbox_max,
                                        resolution=resolution,
                                        aggregation=aggregation_method.value
                                    )
                                elif grid_type_val == 'adaptive' and ADAPTIVE_AVAILABLE:
                                    grid_to_save = AdaptiveResolutionGrid(
                                        bbox_min=bbox_min,
                                        bbox_max=bbox_max,
                                        base_resolution=resolution
                                    )
                                elif grid_type_val == 'multi' and MULTI_AVAILABLE:
                                    grid_to_save = MultiResolutionGrid(
                                        bbox_min=bbox_min,
                                        bbox_max=bbox_max,
                                        base_resolution=resolution,
                                        num_levels=3,
                                        level_ratio=2.0
                                    )
                                else:
                                    # Demo mode
                                    class DemoGrid:
                                        def __init__(self, bbox_min, bbox_max, resolution):
                                            self.bbox_min = np.array(bbox_min)
                                            self.bbox_max = np.array(bbox_max)
                                            self.resolution = resolution
                                            self.size = self.bbox_max - self.bbox_min
                                            self.dims = np.ceil(self.size / resolution).astype(int)
                                            self.dims = np.maximum(self.dims, [1, 1, 1])
                                            self.actual_size = self.dims * resolution
                                            self.aggregation = aggregation_method.value
                                            self.grid_type = grid_type_val
                                    grid_to_save = DemoGrid(bbox_min, bbox_max, resolution)
                    except Exception as e:
                        error_display.value = f"<span style='color: orange;'>⚠️ Error loading alignment {alignment_id[:8]}... for {model_name}: {e}. Using current grid.</span>"
                        grid_to_save = current_grid
                
                # Collect all configuration metadata
                config_metadata = {
                    # Grid type and mode
                    'grid_type': grid_type.value,
                    'resolution_mode': resolution_mode.value,
                    
                    # Resolution settings
                    'uniform_resolution': uniform_resolution.value if resolution_mode.value == 'uniform' else None,
                    'x_resolution': x_resolution.value if resolution_mode.value == 'per_axis' else None,
                    'y_resolution': y_resolution.value if resolution_mode.value == 'per_axis' else None,
                    'z_resolution': z_resolution.value if resolution_mode.value == 'per_axis' else None,
                    
                    # Bounding box mode
                    'bbox_mode': bbox_mode.value,
                    'bbox_source': 'alignment' if bbox_mode.value == 'alignment' else ('model' if bbox_mode.value == 'model' else 'custom'),
                    
                    # Alignment info (if created from alignment)
                    'alignment_id': alignment_id if alignment_id else None,
                    'created_from_alignment': True if alignment_id else False,
                    
                    # Coordinate system
                    'coordinate_system': {
                        'type': coord_system.value,
                        'origin': [origin_x.value, origin_y.value, origin_z.value],
                        'rotation': [rotation_x.value, rotation_y.value, rotation_z.value]
                    },
                    
                    # Grid properties
                    'aggregation_method': aggregation_method.value,
                    'sparse_storage': sparse_storage.value,
                    'compression': compression.value,
                    
                    # Adaptive settings (if applicable)
                    'adaptive_strategy': adaptive_strategy.value if resolution_mode.value == 'adaptive' else None
                }
                
                # Create description
                description = f"{grid_type.value} grid for {model_name}"
                if alignment_id:
                    description += f" (from alignment {alignment_id[:8]}...)"
                
                # Create tags
                tags = [grid_type.value, 'notebook', 'interactive', model_name] if model_name else [grid_type.value, 'notebook', 'interactive']
                if alignment_id:
                    tags.append('aligned')
                
                # Save grid for this model (use grid_to_save which may be alignment-specific)
                grid_id = voxel_storage.save_voxel_grid(
                    model_id=model_id,
                    grid_name=grid_name,
                    voxel_grid=grid_to_save,
                    description=description,
                    tags=tags,
                    model_name=model_name,
                    configuration_metadata=config_metadata
                )
                
                saved_count += 1
                saved_grid_ids.append(grid_id)
                
                # Store in saved_grids
                saved_grids[grid_id] = {
                    'grid_id': grid_id,
                    'grid_name': grid_name,
                    'model_id': model_id,
                    'model_name': model_name,
                    'grid_type': grid_type.value,
                    'created_at': datetime.now()
                }
                
                # Update current grid info if this is the first/only model being saved
                if idx == 0:
                    current_grid_id = grid_id
                    current_grid_model_name = model_name
                    current_grid_model_id = model_id
                
            except Exception as e:
                error_display.value = f"<span style='color: orange;'>⚠️ Error saving grid for {model_name}: {str(e)}</span>"
                continue
        
        progress_bar.value = 100
        
        if saved_count == total_models:
            if total_models > 1:
                status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Saved grids for {saved_count}/{total_models} models</span>"
                error_display.value = f"<span style='color: green;'>✅ Successfully saved {saved_count} grids</span>"
            else:
                status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Grid saved successfully (ID: {saved_grid_ids[0][:8]}...)</span>"
                error_display.value = f"<span style='color: green;'>✅ Saved as '{saved_grids[saved_grid_ids[0]]['grid_name']}' for model {models_to_save[0].get('model_id', '')[:8]}...</span>"
        elif saved_count > 0:
            status_display.value = f"<b>Status:</b> <span style='color: orange;'>⚠️ Saved {saved_count}/{total_models} grids (some failed)</span>"
            error_display.value = f"<span style='color: orange;'>⚠️ Partially saved: {saved_count}/{total_models} grids</span>"
        else:
            status_display.value = f"<b>Status:</b> <span style='color: red;'>Error: Failed to save any grids</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error saving grid: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error saving grid</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

# ============================================
# Load Grid Function
# ============================================

def load_grid(button):
    """Load a grid from MongoDB and display its properties."""
    global current_grid, current_grid_type, current_grid_id, current_grid_model_name, current_grid_model_id
    
    if not voxel_storage or not mongo_client or not mongo_client.is_connected():
        error_display.value = "<span style='color: red;'>❌ MongoDB storage not available. Cannot load grid.</span>"
        status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Storage unavailable</span>"
        return
    
    try:
        status_display.value = "<b>Status:</b> Loading available grids..."
        progress_bar.value = 10
        
        # List all available grids
        available_grids = voxel_storage.list_grids(limit=100)
        
        if not available_grids:
            error_display.value = "<span style='color: orange;'>⚠️ No grids found in database.</span>"
            status_display.value = "<b>Status:</b> No grids available"
            progress_bar.value = 0
            return
        
        progress_bar.value = 30
        
        # Create a simple selection interface
        # For now, use the first grid or let user select
        # In a full implementation, you'd show a dropdown or dialog
        selected_grid = available_grids[0]  # Use first grid for now
        
        grid_id = selected_grid['grid_id']
        status_display.value = f"<b>Status:</b> Loading grid {grid_id[:8]}..."
        progress_bar.value = 50
        
        # Load the grid data
        grid_data = voxel_storage.load_voxel_grid(grid_id)
        
        if not grid_data:
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid {grid_id[:8]}...</span>"
            status_display.value = "<b>Status:</b> <span style='color: red;'>Error loading grid</span>"
            progress_bar.value = 0
            return
        
        progress_bar.value = 70
        
        # Extract metadata
        metadata = grid_data.get('metadata', {})
        
        # Update global state
        current_grid_id = grid_id
        current_grid_model_name = grid_data.get('model_name', 'Unknown')
        current_grid_model_id = grid_data.get('model_id', 'Unknown')
        
        # Get grid type from metadata
        grid_type_from_meta = metadata.get('grid_type', 'uniform')
        if 'VoxelGrid' in str(grid_type_from_meta):
            current_grid_type = 'uniform'
        elif 'Adaptive' in str(grid_type_from_meta):
            current_grid_type = 'adaptive'
        elif 'Multi' in str(grid_type_from_meta):
            current_grid_type = 'multi'
        else:
            current_grid_type = metadata.get('grid_type', 'uniform')
        
        # Create a demo grid object for display (we can't fully reconstruct the grid without the actual voxel data)
        # But we can show the metadata
        class LoadedGridDisplay:
            def __init__(self, metadata):
                self.bbox_min = np.array(metadata.get('bbox_min', [0, 0, 0]))
                self.bbox_max = np.array(metadata.get('bbox_max', [1, 1, 1]))
                self.resolution = metadata.get('resolution', 1.0)
                self.dims = np.array(metadata.get('dims', [1, 1, 1]))
                self.aggregation = metadata.get('aggregation', 'mean')
        
        current_grid = LoadedGridDisplay(metadata)
        
        # Update UI widgets to match loaded grid configuration
        if 'coordinate_system' in metadata:
            coord = metadata['coordinate_system']
            if isinstance(coord, dict):
                if 'type' in coord:
                    coord_system.value = coord['type']
                if 'origin' in coord:
                    origin = coord['origin']
                    if len(origin) >= 3:
                        origin_x.value = origin[0]
                        origin_y.value = origin[1]
                        origin_z.value = origin[2]
                if 'rotation' in coord:
                    rot = coord['rotation']
                    if len(rot) >= 3:
                        rotation_x.value = rot[0]
                        rotation_y.value = rot[1]
                        rotation_z.value = rot[2]
        
        if 'aggregation_method' in metadata:
            aggregation_method.value = metadata['aggregation_method']
        
        if 'sparse_storage' in metadata:
            sparse_storage.value = metadata['sparse_storage']
        
        if 'compression' in metadata:
            compression.value = metadata['compression']
        
        if 'resolution_mode' in metadata:
            resolution_mode.value = metadata['resolution_mode']
            if metadata['resolution_mode'] == 'uniform' and 'uniform_resolution' in metadata:
                uniform_resolution.value = metadata['uniform_resolution']
            elif metadata['resolution_mode'] == 'per_axis':
                if 'x_resolution' in metadata:
                    x_resolution.value = metadata['x_resolution']
                if 'y_resolution' in metadata:
                    y_resolution.value = metadata['y_resolution']
                if 'z_resolution' in metadata:
                    z_resolution.value = metadata['z_resolution']
        
        if 'bbox_mode' in metadata:
            bbox_mode.value = metadata['bbox_mode']
        
        progress_bar.value = 90
        
        # Update displays
        update_grid_displays()
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Grid loaded successfully</span>"
        error_display.value = f"<span style='color: green;'>✅ Loaded grid: {selected_grid.get('grid_name', 'Unknown')} (ID: {grid_id[:8]}...)</span>"
        
    except Exception as e:
        error_display.value = f"<span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error loading grid</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

# Connect events
create_button.on_click(create_grid)
load_button.on_click(load_grid)
save_button.on_click(save_grid)
slice_axis.observe(lambda x: update_slice_view(), names='value')
slice_position.observe(lambda x: update_slice_view(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to create and configure voxel grids.

### Key Takeaways

1. **Grid Types**: Uniform, Adaptive, and Multi-Resolution grids for different use cases
2. **Bounding Box**: Configure spatial extent from models or custom coordinates
3. **Resolution**: Set uniform, per-axis, or adaptive resolution
4. **Coordinate Systems**: Transform between machine, part, and build coordinate systems
5. **Visualization**: View grids in 3D, 2D slices, or property tables

### Next Steps

Proceed to:
- **03_Signal_Mapping_Fundamentals.ipynb** - Learn to map signals to voxel grids
- **04_Temporal_and_Spatial_Alignment.ipynb** - Learn synchronization and alignment

### Related Resources

- Voxelization Module Documentation: `../docs/AM_QADF/05-modules/voxelization.md`
- API Reference: `../docs/AM_QADF/06-api-reference/voxelization-api.md`
- Examples: `../examples/`
